In [1]:
# insert code here
import numpy as np
import pandas as pd
import datetime as dt


%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
#from lifetimes.datasets import load_cdnow_summary
from lifetimes.utils import calibration_and_holdout_data
import seaborn as sns
sns.set(style = 'ticks')

In [ ]:
customer_ds = pd.read_excel('C:/Users/Jewin/Desktop/Jovita-DataSciene/DATA/Capstone/Online Retail.xlsx')
#oder_ds = pd.read_csv('C:/Users/Jewin/Desktop/Jovita-DataSciene/DATA/Capstone/olist_orders_dataset.csv')
#payment_ds = pd.read_csv('C:/Users/Jewin/Desktop/Jovita-DataSciene/DATA/Capstone/olist_order_payments_dataset.csv')

In [ ]:
customer_ds

In [ ]:
print(customer_ds.isnull().sum())

In [ ]:
customer_ds=customer_ds[~customer_ds['CustomerID'].isnull()]

In [ ]:
customer_ds

In [ ]:
customer_ds.dtypes

In [ ]:
#data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
customer_ds['payment_value']=customer_ds['Quantity']*customer_ds['UnitPrice']

In [ ]:
customer_ds.describe().T

In [ ]:
max_date=pd.to_datetime(customer_ds['InvoiceDate'].max(), format=None).to_period('D').to_timestamp()

In [ ]:
max_date

In [ ]:
#!pip install lifetimes

In [ ]:
'''
summary_cal_holdout = calibration_and_holdout_data(customer_ds, 
                                                   customer_id_col = 'CustomerID', 
                                                   datetime_col = 'InvoiceDate', 
                                                   freq = 'D', #days
                                        calibration_period_end='2011-12-09',
                                        observation_period_end='12/9/2011  12:50:00')
summary_cal_holdout
'''

In [ ]:
def cal_RFM(df,customer):
    period_end=pd.to_datetime(df['InvoiceDate'].max(), format=None).to_period('D').to_timestamp()
    #local_df = pd.DataFrame(data[cust_col].unique())
    #local_df.columns = [cust_col]

    #get the max purchase date for each customer and create a dataframe with it
    recency = df.groupby('CustomerID').InvoiceDate.max().reset_index()
    recency.columns = ['CustomerID','MaxOrderdate']

    #we take our observation point as the max invoice date in our dataset
    recency['Recency'] = (recency['MaxOrderdate'].max() - recency['MaxOrderdate']).dt.days
    
    #merge this dataframe to our new user dataframe
    customer = pd.merge(customer, recency[['CustomerID','Recency']], on='CustomerID')
    
    frequency = df.groupby('CustomerID').InvoiceDate.count().reset_index()
    frequency.columns = ['CustomerID','Frequency']
    customer = pd.merge(customer, frequency, on='CustomerID')
    
    payment = df.groupby('CustomerID').payment_value.sum().reset_index()
    payment.columns = ['CustomerID','Payment']
    customer = pd.merge(customer, payment, on='CustomerID')
    return customer

In [ ]:
customer=pd.DataFrame(customer_ds['CustomerID'].unique())
customer.columns = ['CustomerID']
customer=cal_RFM(customer_ds,customer)

In [ ]:
#Testing delete it

'''
customer = pd.DataFrame(customer_ds['CustomerID'].unique())
customer.columns = ['CustomerID']

#get the max purchase date for each customer and create a dataframe with it
max_orderdate = customer_ds.groupby('CustomerID').InvoiceDate.max().reset_index()
max_orderdate.columns = ['CustomerID','MaxOrderdate']

#we take our observation point as the max invoice date in our dataset
max_orderdate['Recency'] = (max_orderdate['MaxOrderdate'].max() - max_orderdate['MaxOrderdate']).dt.days

#merge this dataframe to our new user dataframe
customer = pd.merge(customer, max_orderdate[['CustomerID','Recency']], on='CustomerID')

customer.head(15)

#plot a recency histogram
'''


In [ ]:
plt.hist(x=customer['Recency'], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

In [ ]:
plt.hist(x=customer['Frequency'], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

In [ ]:
'''
frequency = data.groupby('customer_id').order_purchase_timestamp.count().reset_index()
frequency.columns = ['customer_id','Frequency']
customer = pd.merge(customer, frequency, on='customer_id')
'''

In [ ]:
plt.hist(x=customer['Payment'], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

In [ ]:
customer.head()

In [ ]:
customer.describe()

In [ ]:
for c in customer.columns:
    fig, ax = plt.subplots(2, 
                           figsize = (10, 5),
                           sharex = True, 
                           gridspec_kw = {'height_ratios': (0.15, 0.85)})

    ax_box = ax[0]
    ax_box = sns.boxplot(customer[c], ax = ax_box)
    ax_box.set(title = c, yticks = [], xlabel = '')
    sns.despine(ax = ax_box, left = True)

    ax_hist = ax[1]
    ax_hist = sns.distplot(customer[c], ax = ax_hist)
    ax_hist.set(xlabel = '')
    sns.despine(ax = ax_hist)

plt.show()

In [ ]:
customer['Recency'].unique()

In [ ]:
customer['Recency'].hist(bins=5, figsize=[14,6])

In [ ]:
customer['Frequency'].hist(bins=10, figsize=[14,6])

In [ ]:
customer['Payment'].hist(bins=10, figsize=[14,6])

In [ ]:
customer.corr()

In [ ]:
plt.figure(figsize=(8,8))
ax=sns.heatmap(customer.corr(),annot = True,square = True)
ax.set_ylim(6.0, 0)

In [ ]:
sns.pairplot(customer)

In [ ]:
customer['Recency_RANK']=customer['Recency'].rank(method='max')
customer['PCNT_RANK']=customer['Recency'].rank(method='max', pct=True)

In [ ]:
customer['Ordered_Recency_Per']=1-customer['PCNT_RANK']

In [ ]:
customer.head()

In [ ]:
customer['Ordered_Recency_Per'].hist(bins=10, figsize=[14,6])

In [ ]:
customer['Frequency_RANK']=customer['Frequency'].rank(method='max', pct=True)

In [ ]:
customer['Payement_RANK']=customer['Payment'].rank(method='max', pct=True)

In [ ]:
customer.head()

In [ ]:
customer['Total_Rank']=(customer['Frequency_RANK']+customer['Payement_RANK']+customer['Ordered_Recency_Per'])/3

In [ ]:
customer.head()

In [ ]:
customer['Total_Rank'].hist(bins=10, figsize=[14,6])

In [ ]:
customer['CLTV']=pd.cut(customer.Total_Rank, bins=4, labels=np.arange(4), right=False)

In [ ]:
customer.head()

In [ ]:
from sklearn.preprocessing import normalize


In [ ]:
df=customer[['CustomerID','Ordered_Recency_Per','Frequency_RANK','Payement_RANK']]

In [ ]:
Xs = normalize(df)

In [ ]:
Xs

In [ ]:
from sklearn.cluster import KMeans



In [ ]:
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(Xs)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
k = 4
# 2 Clusters
k_mean2 = KMeans(n_clusters=k)
k_mean2.fit(Xs)

In [ ]:
customer['Cluster'] = k_mean2.labels_

In [ ]:
customer

In [ ]:
customer['Cluster'].value_counts()

In [ ]:
customer['CLTV'].value_counts()

In [ ]:
sns.lmplot(x='Recency', y='Frequency', data=customer, hue='Cluster', 
           fit_reg=False)

In [ ]:
sns.lmplot(x='Payment', y='Frequency', data=customer, hue='Cluster', 
           fit_reg=False)

In [ ]:
sns.scatterplot(x="Recency", y="Frequency",
                     hue="Payment", style="Cluster", data=customer)

In [ ]:
ax = sns.scatterplot(x="Recency", y="Frequency",
                     hue="Payment", size="Cluster",
                     data=customer)

# Predicting next purchase date

In [ ]:
Final_end=pd.to_datetime(customer_ds['InvoiceDate'].max(), format=None).to_period('D').to_timestamp()

In [ ]:
start_date=pd.to_datetime(customer_ds['InvoiceDate'].min(), format=None).to_period('D').to_timestamp()

In [ ]:
from datetime import date,timedelta

In [ ]:
split_date=date(2011,8,1)

In [ ]:
end_date=date(2011,11,1)

In [ ]:
customer_ds

In [ ]:
customer_ds.isnull().sum()

In [ ]:
training_period=customer_ds.loc[(customer_ds.InvoiceDate >= start_date) & (customer_ds.InvoiceDate <= split_date)]
testing_period=customer_ds.loc[(customer_ds.InvoiceDate > split_date) & (customer_ds.InvoiceDate <= end_date)]

In [ ]:
data=pd.DataFrame(training_period['CustomerID'].unique())
data.columns = ['CustomerID']

In [ ]:
data

In [ ]:
#first purchase in testing period
testing_period_first_purchase = testing_period.groupby('CustomerID').InvoiceDate.min().reset_index()
testing_period_first_purchase.columns = ['CustomerID','MinPurchaseDate']

#Last purchase date in training period
training_period_last_purchase = training_period.groupby('CustomerID').InvoiceDate.max().reset_index()
training_period_last_purchase.columns = ['CustomerID','MaxPurchaseDate']

In [ ]:
testing_period_first_purchase

In [ ]:
training_period_last_purchase

In [ ]:
train_test_merge=pd.merge(training_period_last_purchase,testing_period_first_purchase,on='CustomerID',how='left')

In [ ]:
train_test_merge

In [ ]:
#n+1 - n
train_test_merge['NextPurchaseDay_TestPeriod'] = (train_test_merge['MinPurchaseDate'] - train_test_merge['MaxPurchaseDate']).dt.days

In [ ]:
data=pd.merge(data, train_test_merge[['CustomerID','NextPurchaseDay_TestPeriod']],on='CustomerID',how='left')

In [ ]:
data
#NaN means they do not have a purchase date in that period . 
#for analysis we could consider it as a large next purchase date and fill that number by 999

In [ ]:
data = data.fillna(999)

In [ ]:
df_Invoice_Day = training_period[['CustomerID','InvoiceDate']]

In [ ]:
df_Invoice_Day

In [ ]:
df_Invoice_Day['InvoiceDay'] = training_period['InvoiceDate'].dt.date

In [ ]:
df_Invoice_Day

In [ ]:
df_Invoice_Day = df_Invoice_Day.sort_values(['CustomerID','InvoiceDate'])

In [ ]:
df_Invoice_Day

In [ ]:
#drop duplicates
df_Invoice_Day = df_Invoice_Day.drop_duplicates(subset=['CustomerID','InvoiceDay'],keep='first')

In [ ]:
df_Invoice_Day

In [ ]:
#shifting last 3 purchase dates
df_Invoice_Day['n-1'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(1)
df_Invoice_Day['n-2'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(2)
df_Invoice_Day['n-3'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(3)
#df_Invoice_Day['n-4'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(4)
#df_Invoice_Day['n-5'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(5)
#df_Invoice_Day['n-6'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(6)
#df_Invoice_Day['n-7'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(7)
#df_Invoice_Day['n-8'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(8)

In [ ]:
df_Invoice_Day

In [ ]:
df_Invoice_Day.notnull().sum()

CustomerID     10674
InvoiceDate    10674
InvoiceDay     10674
n-1             7481
n-2             5534
n-3             4248
n-4             3384
n-5             2755
n-6             2275
n-7             1902
n-8             1626
dtype: int64

In [ ]:
df_Invoice_Day['DayDiff1'] = (df_Invoice_Day['InvoiceDay'] - df_Invoice_Day['n-1']).dt.days
df_Invoice_Day['DayDiff2'] = (df_Invoice_Day['n-1'] - df_Invoice_Day['n-2']).dt.days
df_Invoice_Day['DayDiff3'] = (df_Invoice_Day['n-2'] - df_Invoice_Day['n-3']).dt.days

In [ ]:
df_Invoice_Day

In [ ]:
df_Invoice_Day.notnull().sum()

In [ ]:
#tx_day_diff = df_Invoice_Day.groupby('CustomerID').agg({'DayDiff1': ['mean']}).reset_index()
#tx_day_diff.columns = ['CustomerID', 'Average_days']

In [ ]:
#tx_day_diff

In [ ]:
#df_Invoice_Day=df_Invoice_Day.drop(columns=['n-4','n-5','n-6','n-7','n-8'])

In [ ]:
day_order_last = df_Invoice_Day.drop_duplicates(subset=['CustomerID'],keep='last')

In [ ]:
day_order_last

In [ ]:
day_order_last = day_order_last.dropna()

In [ ]:
day_order_last

In [ ]:
data

In [ ]:
data = pd.merge(data, day_order_last[['CustomerID','InvoiceDay','DayDiff1','DayDiff2','DayDiff3']], on='CustomerID')

In [ ]:
data

In [ ]:
data['Average']=(data['DayDiff1']+data['DayDiff2']+data['DayDiff3'])/3

In [ ]:
data

In [ ]:
data['NextPurchaseDayRange'] = 4
data.loc[data.NextPurchaseDay_TestPeriod<20,'NextPurchaseDayRange'] = 0
data.loc[data.NextPurchaseDay_TestPeriod<50,'NextPurchaseDayRange'] = 1
data.loc[data.NextPurchaseDay_TestPeriod<100,'NextPurchaseDayRange'] = 2
data.loc[data.NextPurchaseDay_TestPeriod<150,'NextPurchaseDayRange'] = 3

In [ ]:
data

In [ ]:
import itertools
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from sklearn import datasets

from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions

In [ ]:
#!pip install mlxtend

In [ ]:
data.corr()

In [ ]:
sns.pairplot(data)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(data.corr())

In [ ]:
X=data[['CustomerID','DayDiff1','DayDiff2','DayDiff3','Average']]
y=data['NextPurchaseDayRange']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits = 10, random_state = 7)
cart = DecisionTreeClassifier()
cart.fit(X_train, y_train)
results = cross_val_score(cart, X_train, y_train, cv = kfold)
print('Decision Tree, Cross-Validation mean: %.4f' % results.mean())

In [ ]:
print('Training Score: ',cart.score(X_train,y_train))
print('Test Score: ',cart.score(X_test,y_test))

In [ ]:
num_trees = 100
model1 = BaggingClassifier(base_estimator = cart, n_estimators = num_trees, random_state = 7)
model1.fit(X_train, y_train)
results = cross_val_score(model1, X_train, y_train, cv = kfold)
print('Bagging, Cross-Validation mean: %.4f' % results.mean())

In [ ]:
print('Training Score: ',model1.score(X_train,y_train))
print('Test Score: ',model1.score(X_test,y_test))

In [ ]:
def data_prep(data,start_date,end_date,split_date):
    training_period=customer_ds.loc[(customer_ds.InvoiceDate >= start_date) & (customer_ds.InvoiceDate <= split_date)]
    testing_period=customer_ds.loc[(customer_ds.InvoiceDate > split_date) & (customer_ds.InvoiceDate <= end_date)]
    data=pd.DataFrame(training_period['CustomerID'].unique())
    data.columns = ['CustomerID']
    #first purchase in testing period
    testing_period_first_purchase = testing_period.groupby('CustomerID').InvoiceDate.min().reset_index()
    testing_period_first_purchase.columns = ['CustomerID','MinPurchaseDate']

    #Last purchase date in training period
    training_period_last_purchase = training_period.groupby('CustomerID').InvoiceDate.max().reset_index()
    training_period_last_purchase.columns = ['CustomerID','MaxPurchaseDate']
    train_test_merge=pd.merge(training_period_last_purchase,testing_period_first_purchase,on='CustomerID',how='left')
    #n+1 - n
    train_test_merge['NextPurchaseDay_TestPeriod'] = (train_test_merge['MinPurchaseDate'] - train_test_merge['MaxPurchaseDate']).dt.days
    data=pd.merge(data, train_test_merge[['CustomerID','NextPurchaseDay_TestPeriod']],on='CustomerID',how='left')
    #NaN means they do not have a purchase date in that period . 
    #for analysis we could consider it as a large next purchase date and fill that number by 999
    data = data.fillna(999)
    df_Invoice_Day = training_period[['CustomerID','InvoiceDate']]
    df_Invoice_Day['InvoiceDay'] = training_period['InvoiceDate'].dt.date
    df_Invoice_Day = df_Invoice_Day.sort_values(['CustomerID','InvoiceDate'])
    df_Invoice_Day = df_Invoice_Day.drop_duplicates(subset=['CustomerID','InvoiceDay'],keep='first')
    #shifting last 3 purchase dates
    df_Invoice_Day['n-1'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(1)
    df_Invoice_Day['n-2'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(2)
    df_Invoice_Day['n-3'] = df_Invoice_Day.groupby('CustomerID')['InvoiceDay'].shift(3)
    df_Invoice_Day['DayDiff1'] = (df_Invoice_Day['InvoiceDay'] - df_Invoice_Day['n-1']).dt.days
    df_Invoice_Day['DayDiff2'] = (df_Invoice_Day['n-1'] - df_Invoice_Day['n-2']).dt.days
    df_Invoice_Day['DayDiff3'] = (df_Invoice_Day['n-2'] - df_Invoice_Day['n-3']).dt.days
    day_order_last = df_Invoice_Day.drop_duplicates(subset=['CustomerID'],keep='last')
    day_order_last = day_order_last.dropna()
    data = pd.merge(data, day_order_last[['CustomerID','InvoiceDay','DayDiff1','DayDiff2','DayDiff3']], on='CustomerID')
    data['Average']=(data['DayDiff1']+data['DayDiff2']+data['DayDiff3'])/3
    data['NextPurchaseDayRange'] = 4
    data.loc[data.NextPurchaseDay_TestPeriod<20,'NextPurchaseDayRange'] = 0
    data.loc[data.NextPurchaseDay_TestPeriod<50,'NextPurchaseDayRange'] = 1
    data.loc[data.NextPurchaseDay_TestPeriod<100,'NextPurchaseDayRange'] = 2
    data.loc[data.NextPurchaseDay_TestPeriod<150,'NextPurchaseDayRange'] = 3
    return data
    
    

In [ ]:
check_data=data_prep(data,start_date,Final_end,end_date)

In [ ]:
check_data

In [ ]:
X=check_data[['CustomerID','DayDiff1','DayDiff2','DayDiff3','Average']]
y=check_data['NextPurchaseDayRange']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
kfold = KFold(n_splits = 10, random_state = 7)
cart = DecisionTreeClassifier()
cart.fit(X_train, y_train)
results = cross_val_score(cart, X_train, y_train, cv = kfold)
print('Decision Tree, Cross-Validation mean: %.4f' % results.mean())
print('Training Score: ',cart.score(X_train,y_train))
print('Test Score: ',cart.score(X_test,y_test))

In [ ]:
num_trees = 100
model1 = BaggingClassifier(base_estimator = cart, n_estimators = num_trees, random_state = 7)
model1.fit(X_train, y_train)
results = cross_val_score(model1, X_train, y_train, cv = kfold)
print('Bagging, Cross-Validation mean: %.4f' % results.mean())
print('Training Score: ',model1.score(X_train,y_train))
print('Test Score: ',model1.score(X_test,y_test))